<a href="https://colab.research.google.com/github/mary270/Computer_Vision/blob/main/shoe_boots_sandals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import zipfile

with zipfile.ZipFile('/content/shoe_boot_sanda_2.zip', 'r') as zip_ref:
    zip_ref.extractall('images')

image_files = [file for file in os.listdir('images') if file.endswith('.jpg')]


In [ ]:
import cv2

model_path = '/content/frozen_inference_graph.pb'

model = cv2.dnn.readNetFromTensorflow(model_path)


In [9]:
def detect_objects(image_path):
    image = cv2.imread(image_path)
    (h, w) = image.shape[:2]

    blob = cv2.dnn.blobFromImage(image, swapRB=True, crop=False)
    model.setInput(blob)
    detections = model.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            class_id = int(detections[0, 0, i, 1])

            if class_id == 1:
                label = 'boots'
            elif class_id == 2:
                label = 'sandals'
            elif class_id == 3:
                label = 'shoes'
            else:
                continue

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')

            crop_img = image[startY:endY, startX:endX]
            cv2.imwrite(f'{label}/{os.path.basename(image_path)}', crop_img)


In [10]:
import os

for file in image_files:
    detect_objects(f'images/{file}')


In [11]:
for label in ['boots', 'sandals', 'shoes']:
    os.makedirs(label, exist_ok=True)

for file in os.listdir():
    if file.endswith('.jpg'):
        os.rename(file, f'{label}/{file}')
